In [21]:
import lmdb
import numpy as np
import cv2
from caffe.proto.caffe_pb2 import Datum
from tqdm import tqdm
import os

In [22]:
env = lmdb.open('./mnist_train_lmdb/', map_size=80000*28*28*5)

In [23]:
img_dir = '../../buff_benchmark/red_number_new/'
no_dig_dir = './no_dig_img/'

In [24]:
with env.begin() as txn:
    print txn.stat()

{'branch_pages': 77L, 'leaf_pages': 16668L, 'overflow_pages': 0L, 'psize': 4096L, 'depth': 3L, 'entries': 66977L}


In [29]:
with env.begin(write=True) as txn:
    i = 60000
    for img_name in tqdm(os.listdir(img_dir)):
        if not img_name.endswith('.jpg'):
            continue
        img_path = os.path.join(img_dir, img_name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        datum = Datum()
        datum.data = img.tobytes()
        datum.width = 28
        datum.height = 28
        datum.channels = 1
        datum.label = int(img_name.split('_')[1].split('.')[0])
        txn.put('{:08}'.format(i), datum.SerializeToString())
        i += 1
    for img_name in tqdm(os.listdir(no_dig_dir)):
        if not img_name.endswith('.jpg'):
            continue
        img_path = os.path.join(no_dig_dir, img_name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        datum = Datum()
        datum.data = img.tobytes()
        datum.width = 28
        datum.height = 28
        datum.channels = 1
        datum.label = 10
        txn.put('{:08}'.format(i), datum.SerializeToString())
        i += 1
    env_new = lmdb.open('./mnist_train_new_lmdb/', map_size=80000*28*28*5)
    rand_ind = np.arange(i)
    np.random.shuffle(rand_ind)
    with env_new.begin(write=True) as txn_new:
        cursor = txn.cursor()
        for key, value in cursor:
            key_idx = int(key)
            datum = Datum()
            datum.ParseFromString(value)
            txn_new.put('{:08}'.format(rand_ind[key_idx]), datum.SerializeToString())
        

100%|██████████| 4929/4929 [00:00<00:00, 9483.04it/s]


In [30]:
with env.begin() as txn:
    print txn.stat()
with env_new.begin() as txn:
    print txn.stat()

{'branch_pages': 77L, 'leaf_pages': 16668L, 'overflow_pages': 0L, 'psize': 4096L, 'depth': 3L, 'entries': 66977L}
{'branch_pages': 130L, 'leaf_pages': 19873L, 'overflow_pages': 0L, 'psize': 4096L, 'depth': 3L, 'entries': 66977L}


In [20]:
a = np.arange(5)
np.random.shuffle(a)
a

array([4, 3, 2, 0, 1])